In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os

In [3]:
input_file = 'advcy_data_final 2.csv'


def load_dataset(file_path):
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        print(f"Dataset '{file_path}' loaded successfully.\n")
        return df
    else:
        raise FileNotFoundError(f"The file '{file_path}' does not exist. Please check the file path.")

# Load the dataset
df = load_dataset(input_file)


Dataset 'advcy_data_final 2.csv' loaded successfully.



In [9]:
# Display the first five rows of the dataset
print("First five rows of the dataset:")
display(df.head())


First five rows of the dataset:


,sustainer,gender,client_state_rank,all_state_rank,client_zip_rank,all_zip_rank,age,account_id,gift_date,gift_min_t12m,...,drtv_t12m,other_t12m,no_channel_t12m,member_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
0,0,0,36,29,3553,34364,0.0,1030075C,2020-03-23,1.0,...,0,0,0,0,1.0,2.0,1.5,2,2,2
1,0,3,33,41,2,22870,0.0,1067737C,2019-09-09,2.0,...,0,0,0,0,2.0,2.0,2.0,1,1,1
2,0,0,42,31,2594,35599,0.0,1075779C,2020-03-23,2.0,...,0,0,0,0,2.0,2.0,2.0,1,1,1
3,0,0,42,31,1394,29773,0.0,1090840C,2020-06-12,2.0,...,0,0,0,0,2.0,2.0,2.0,1,1,1
4,0,0,42,31,581,28865,0.0,1155437C,2020-11-02,2.0,...,0,0,0,0,2.0,2.0,2.0,1,1,1


In [10]:
# Define the list of columns you want to drop
columns_to_drop = ['drtv_t12m', 'no_channel_t12m', 'member_t12m', 'f2f_t12m','account_id', 'other_t12m','events_t12m', 'age','sustainer' ] 
# all values are single values 

print(f"Columns intended to drop: {columns_to_drop}\n")

existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]

if existing_columns_to_drop:
    print(f"Columns to be dropped (exist in DataFrame): {existing_columns_to_drop}\n")
else:
    print("No specified columns found in the DataFrame to drop.\n")
df = df.drop(columns=existing_columns_to_drop)
print(f"Successfully dropped columns: {existing_columns_to_drop}\n")

print("Remaining columns after dropping:")
print(df.columns.tolist())


Columns intended to drop: ['drtv_t12m', 'no_channel_t12m', 'member_t12m', 'f2f_t12m', 'account_id', 'other_t12m', 'events_t12m', 'age', 'sustainer']

Columns to be dropped (exist in DataFrame): ['drtv_t12m', 'no_channel_t12m', 'member_t12m', 'f2f_t12m', 'account_id', 'other_t12m', 'events_t12m', 'age', 'sustainer']

Successfully dropped columns: ['drtv_t12m', 'no_channel_t12m', 'member_t12m', 'f2f_t12m', 'account_id', 'other_t12m', 'events_t12m', 'age', 'sustainer']

Remaining columns after dropping:
['gender', 'client_state_rank', 'all_state_rank', 'client_zip_rank', 'all_zip_rank', 'gift_date', 'gift_min_t12m', 'gift_max_t12m', 'gift_avg_t12m', 'gift_count_t12m', 'promo_count_t12m', 'cons_month_t12m', 'dm_t12m', 'tm_t12m', 'wm_t12m', 'digital_t12m', 'gift_min_lt', 'gift_max_lt', 'gift_avg_lt', 'gift_count_lt', 'promo_count_lt', 'cons_month_lt']


In [11]:

def handle_missing_values(df, strategy='fill', fill_value=0):
    if strategy == 'fill':
        df_filled = df.fillna(fill_value)
        print(f"Missing values filled with {fill_value}.\n")
        return df_filled
    elif strategy == 'drop':
        df_dropped = df.dropna()
        print("Rows with missing values have been dropped.\n")
        return df_dropped
    else:
        raise ValueError("Unsupported strategy for handling missing values.")

# Handle missing values by filling with 0
df = handle_missing_values(df, strategy='fill', fill_value=0)

Missing values filled with 0.



In [12]:
numerical_columns = df.select_dtypes(include=[np.number]).columns
print("Numerical columns:", numerical_columns.tolist())


Numerical columns: ['gender', 'client_state_rank', 'all_state_rank', 'client_zip_rank', 'all_zip_rank', 'gift_min_t12m', 'gift_max_t12m', 'gift_avg_t12m', 'gift_count_t12m', 'promo_count_t12m', 'cons_month_t12m', 'dm_t12m', 'tm_t12m', 'wm_t12m', 'digital_t12m', 'gift_min_lt', 'gift_max_lt', 'gift_avg_lt', 'gift_count_lt', 'promo_count_lt', 'cons_month_lt']


Detecting Outliers We'll detect outliers using two methods:

Z-Score Method, Interquartile Range (IQR) Method

In [13]:
# Function to detect outliers using Z-Score
def detect_outliers_zscore(df, numerical_columns, threshold=3):
    outliers = {}
    print("Detecting outliers using Z-Score method\n")
    for column in numerical_columns:
        if df[column].nunique() > 1:
            z_scores = np.abs(stats.zscore(df[column]))
            outliers_indices = np.where(z_scores > threshold)[0]
            outliers[column] = outliers_indices
            print(f"Z-Score Outliers in '{column}': {len(outliers_indices)} detected")
        else:
            outliers[column] = np.array([])
            print(f"Z-Score Outliers in '{column}': 0 detected (constant column)")
    print("\n")
    return outliers

# Detect outliers using Z-Score
outliers_zscore = detect_outliers_zscore(df, numerical_columns)


Detecting outliers using Z-Score method

Z-Score Outliers in 'gender': 0 detected
Z-Score Outliers in 'client_state_rank': 223 detected
Z-Score Outliers in 'all_state_rank': 194 detected
Z-Score Outliers in 'client_zip_rank': 0 detected
Z-Score Outliers in 'all_zip_rank': 486 detected
Z-Score Outliers in 'gift_min_t12m': 100 detected
Z-Score Outliers in 'gift_max_t12m': 52 detected
Z-Score Outliers in 'gift_avg_t12m': 46 detected
Z-Score Outliers in 'gift_count_t12m': 200 detected
Z-Score Outliers in 'promo_count_t12m': 220 detected
Z-Score Outliers in 'cons_month_t12m': 525 detected
Z-Score Outliers in 'dm_t12m': 0 detected
Z-Score Outliers in 'tm_t12m': 671 detected
Z-Score Outliers in 'wm_t12m': 0 detected
Z-Score Outliers in 'digital_t12m': 0 detected
Z-Score Outliers in 'gift_min_lt': 109 detected
Z-Score Outliers in 'gift_max_lt': 38 detected
Z-Score Outliers in 'gift_avg_lt': 94 detected
Z-Score Outliers in 'gift_count_lt': 293 detected
Z-Score Outliers in 'promo_count_lt': 405 

In [15]:
# Function to detect outliers using IQR
def detect_outliers_iqr(df, numerical_columns):
    outliers = {}
    print("Detecting outliers using IQR method...\n")
    for column in numerical_columns:
        Q1 = df[column].quantile(0.10)
        Q3 = df[column].quantile(0.90)
        IQR = Q3 - Q1
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        condition = (df[column] < lower_bound) | (df[column] > upper_bound)
        outliers_indices = df[condition].index
        outliers[column] = outliers_indices
        print(f"IQR Outliers in '{column}': {len(outliers_indices)} detected")
    print("\n")
    return outliers

# Detect outliers using IQR
outliers_iqr = detect_outliers_iqr(df, numerical_columns)

Detecting outliers using IQR method...

IQR Outliers in 'gender': 0 detected
IQR Outliers in 'client_state_rank': 0 detected
IQR Outliers in 'all_state_rank': 0 detected
IQR Outliers in 'client_zip_rank': 0 detected
IQR Outliers in 'all_zip_rank': 2 detected
IQR Outliers in 'gift_min_t12m': 204 detected
IQR Outliers in 'gift_max_t12m': 418 detected
IQR Outliers in 'gift_avg_t12m': 262 detected
IQR Outliers in 'gift_count_t12m': 0 detected
IQR Outliers in 'promo_count_t12m': 0 detected
IQR Outliers in 'cons_month_t12m': 1 detected
IQR Outliers in 'dm_t12m': 0 detected
IQR Outliers in 'tm_t12m': 671 detected
IQR Outliers in 'wm_t12m': 0 detected
IQR Outliers in 'digital_t12m': 0 detected
IQR Outliers in 'gift_min_lt': 139 detected
IQR Outliers in 'gift_max_lt': 410 detected
IQR Outliers in 'gift_avg_lt': 168 detected
IQR Outliers in 'gift_count_lt': 39 detected
IQR Outliers in 'promo_count_lt': 0 detected
IQR Outliers in 'cons_month_lt': 37 detected




Removing Outliers Based on IQR We'll remove the detected outliers using the IQR method to obtain a dataset without outliers

In [17]:
def remove_outliers_iqr(df, numerical_columns):
    print("Removing outliers based on IQR method...\n")
    Q1 = df[numerical_columns].quantile(0.10)
    Q3 = df[numerical_columns].quantile(0.90)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    condition = ~((df[numerical_columns] < lower_bound) | (df[numerical_columns] > upper_bound)).any(axis=1)
    df_no_outliers = df[condition]
    print("Outliers removed.")
    print(f"Original dataset size: {df.shape[0]} rows")
    print(f"Dataset without outliers size: {df_no_outliers.shape[0]} rows\n")
    return df_no_outliers

# Remove outliers using IQR
df_without_outliers = remove_outliers_iqr(df, numerical_columns)


Removing outliers based on IQR method...

Outliers removed.
Original dataset size: 19573 rows
Dataset without outliers size: 18167 rows



In [18]:

def display_dataset_without_outliers(df):
    print("Displaying the dataset without outliers:\n")
    # Display the first 5 rows
    print("First five rows of the dataset without outliers:")
    display(df.head())
    
    print("\nSummary Statistics of Dataset Without Outliers:")
    display(df.describe())
    
    # Display the number of rows and columns
    print(f"\nDataset without outliers contains {df.shape[0]} rows and {df.shape[1]} columns.\n")

display_dataset_without_outliers(df_without_outliers)


Displaying the dataset without outliers:

First five rows of the dataset without outliers:


,gender,client_state_rank,all_state_rank,client_zip_rank,all_zip_rank,gift_date,gift_min_t12m,gift_max_t12m,gift_avg_t12m,gift_count_t12m,...,dm_t12m,tm_t12m,wm_t12m,digital_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
0,0,36,29,3553,34364,2020-03-23,1.0,2.0,1.5,2,...,1,0,0,0,1.0,2.0,1.5,2,2,2
1,3,33,41,2,22870,2019-09-09,2.0,2.0,2.0,1,...,1,0,0,0,2.0,2.0,2.0,1,1,1
2,0,42,31,2594,35599,2020-03-23,2.0,2.0,2.0,1,...,1,0,0,0,2.0,2.0,2.0,1,1,1
3,0,42,31,1394,29773,2020-06-12,2.0,2.0,2.0,1,...,1,0,0,0,2.0,2.0,2.0,1,1,1
4,0,42,31,581,28865,2020-11-02,2.0,2.0,2.0,1,...,1,0,0,0,2.0,2.0,2.0,1,1,1



Summary Statistics of Dataset Without Outliers:


,gender,client_state_rank,all_state_rank,client_zip_rank,all_zip_rank,gift_min_t12m,gift_max_t12m,gift_avg_t12m,gift_count_t12m,promo_count_t12m,...,dm_t12m,tm_t12m,wm_t12m,digital_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
count,18167.000000,18167.000000,18167.000000,18167.000000,18167.000000,18167.000000,18167.000000,18167.000000,18167.000000,18167.000000,...,18167.000000,18167.0,18167.000000,18167.000000,18167.000000,18167.000000,18167.000000,18167.000000,18167.000000,18167.000000
mean,0.864204,40.133098,38.599439,2908.767546,33541.197281,35.763384,53.757579,42.575190,3.999725,3.324434,...,0.769637,0.0,0.170089,0.436010,22.052789,73.602853,37.254292,12.090274,10.623218,4.477019
std,1.255484,10.811375,10.905402,1438.017856,5004.592227,35.798015,52.166043,39.722352,4.095665,3.375144,...,0.421077,0.0,0.375721,0.495902,18.037024,75.023338,28.521122,13.297397,11.514027,5.633794
min,0.000000,1.000000,1.000000,2.000000,1738.000000,0.500000,1.000000,1.000000,1.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.500000,1.000000,1.000000,1.000000,0.000000,1.000000
25%,0.000000,34.000000,32.000000,1679.000000,32420.000000,15.000000,20.000000,19.440000,1.000000,1.000000,...,1.000000,0.0,0.000000,0.000000,10.000000,25.000000,20.000000,3.000000,2.000000,1.000000
50%,0.000000,43.000000,42.000000,3212.000000,35291.000000,25.000000,35.000000,28.330000,2.000000,2.000000,...,1.000000,0.0,0.000000,0.000000,20.000000,50.000000,28.330000,7.000000,6.000000,2.000000
75%,1.000000,49.000000,47.000000,4224.000000,36641.000000,50.000000,75.000000,50.000000,5.000000,4.000000,...,1.000000,0.0,0.000000,1.000000,25.000000,100.000000,46.920000,17.000000,15.000000,6.000000
max,3.000000,51.000000,51.000000,4794.000000,37517.000000,350.000000,450.000000,350.000000,25.000000,22.000000,...,1.000000,0.0,1.000000,1.000000,111.000000,725.000000,266.670000,113.000000,83.000000,45.000000



Dataset without outliers contains 18167 rows and 22 columns.



In [19]:
def save_dataset_without_outliers(df, output_path):
    try:
        df.to_csv(output_path, index=False)
        print(f"Dataset without outliers saved to '{output_path}'.\n")
    except Exception as e:
        print(f"Error saving file: {e}\n")


output_file = 'advcydata_without_outliers.csv'

# Save the cleaned dataset
save_dataset_without_outliers(df_without_outliers, output_file)


Dataset without outliers saved to 'advcydata_without_outliers.csv'.

